# 💳 Stripe Utilities Tests

> Tests for payment processing, subscription management, and access control.

In [ ]:
#| hide
import os
os.environ['DB_TYPE'] = 'SQLITE'
os.environ['DB_NAME'] = 'test_stripe'
os.environ['CONFIG_STRIPE_SECRETKEY'] = 'sk_test_fake_key_for_testing'
os.environ['CONFIG_STRIPE_WEBHOOKSECRET'] = 'whsec_test_fake_secret'
os.environ['CONFIG_STRIPE_MONTHLY_PRICE_ID'] = 'price_monthly_test'
os.environ['CONFIG_STRIPE_YEARLY_PRICE_ID'] = 'price_yearly_test'
os.environ['CONFIG_STRIPE_BASE_URL'] = 'http://localhost:5001'
os.environ['ENVIRONMENT'] = 'development'

In [ ]:
from fh_saas.db_host import HostDatabase, Subscription, gen_id, timestamp
from fh_saas.utils_stripe import (
    StripeConfig,
    StripeService,
    get_active_subscription,
    has_active_subscription,
    require_active_subscription,
    get_subscription_status,
    check_feature_access,
    require_feature_access,
    get_stripe_service,
    reset_stripe_service,
    TIER_HIERARCHY,
)
from datetime import datetime, timedelta
from unittest.mock import MagicMock, patch
import json

In [ ]:
#| hide

# Cleanup function for idempotent tests
def cleanup_test_db():
    """Reset database state for clean tests."""
    import os
    # Reset singletons
    HostDatabase.reset_instance()
    reset_stripe_service()
    
    # Remove test database file
    db_file = 'test_stripe.db'
    if os.path.exists(db_file):
        os.remove(db_file)

cleanup_test_db()

## StripeConfig Tests

In [ ]:
print("🧪 Testing StripeConfig...")

# Test 1: from_env() loads configuration
config = StripeConfig.from_env()
assert config.secret_key == 'sk_test_fake_key_for_testing'
assert config.webhook_secret == 'whsec_test_fake_secret'
assert config.monthly_price_id == 'price_monthly_test'
assert config.yearly_price_id == 'price_yearly_test'
assert config.trial_days == 30
assert config.grace_period_days == 3
assert config.is_development == True
print("   ✅ from_env() loads all configuration")

# Test 2: success_url and cancel_url properties
assert '{CHECKOUT_SESSION_ID}' in config.success_url
assert config.base_url in config.cancel_url
print("   ✅ URL properties work correctly")

# Test 3: Default feature tiers
assert 'advanced_analytics' in config.feature_tiers
assert config.feature_tiers['advanced_analytics'] == 'yearly'
print("   ✅ Default feature tiers configured")

# Test 4: Manual config creation
manual_config = StripeConfig(
    secret_key='sk_manual',
    trial_days=14,
    grace_period_days=7,
)
assert manual_config.trial_days == 14
assert manual_config.grace_period_days == 7
print("   ✅ Manual config creation works")

print("\n✅ All StripeConfig tests PASSED!")

## Subscription Access Control Tests

In [ ]:
print("🧪 Testing Subscription Access Control...")

# Setup test database
host_db = HostDatabase.from_env()

# Create test subscriptions
tenant_active = 'tnt_active_test'
tenant_trialing = 'tnt_trialing_test'
tenant_past_due = 'tnt_pastdue_test'
tenant_canceled = 'tnt_canceled_test'
tenant_no_sub = 'tnt_nosub_test'

# Active subscription
sub_active = Subscription(
    id=gen_id(),
    tenant_id=tenant_active,
    stripe_sub_id='sub_active_123',
    stripe_cust_id='cus_123',
    plan_tier='monthly',
    status='active',
    current_period_end=(datetime.utcnow() + timedelta(days=30)).isoformat(),
    payment_type='subscription',
    created_at=timestamp(),
)
host_db.subscriptions.insert(sub_active)

# Trialing subscription
sub_trialing = Subscription(
    id=gen_id(),
    tenant_id=tenant_trialing,
    stripe_sub_id='sub_trial_123',
    stripe_cust_id='cus_456',
    plan_tier='yearly',
    status='trialing',
    current_period_end=(datetime.utcnow() + timedelta(days=30)).isoformat(),
    trial_end=(datetime.utcnow() + timedelta(days=14)).isoformat(),
    payment_type='subscription',
    created_at=timestamp(),
)
host_db.subscriptions.insert(sub_trialing)

# Past due subscription (within grace period)
sub_past_due = Subscription(
    id=gen_id(),
    tenant_id=tenant_past_due,
    stripe_sub_id='sub_pastdue_123',
    stripe_cust_id='cus_789',
    plan_tier='monthly',
    status='past_due',
    current_period_end=(datetime.utcnow() + timedelta(days=1)).isoformat(),  # Still in grace
    payment_type='subscription',
    created_at=timestamp(),
)
host_db.subscriptions.insert(sub_past_due)

# Canceled subscription
sub_canceled = Subscription(
    id=gen_id(),
    tenant_id=tenant_canceled,
    stripe_sub_id='sub_canceled_123',
    stripe_cust_id='cus_000',
    plan_tier='monthly',
    status='canceled',
    current_period_end=(datetime.utcnow() - timedelta(days=10)).isoformat(),
    cancel_at_period_end=True,
    payment_type='subscription',
    created_at=timestamp(),
)
host_db.subscriptions.insert(sub_canceled)

host_db.commit()
print("   📊 Test data created")

In [ ]:
# Test get_active_subscription
print("\n🧪 Testing get_active_subscription...")

# Active subscription
result = get_active_subscription(tenant_active, host_db)
assert result is not None, "Should find active subscription"
assert result.status == 'active'
print("   ✅ Active subscription found")

# Trialing subscription
result = get_active_subscription(tenant_trialing, host_db)
assert result is not None, "Should find trialing subscription"
assert result.status == 'trialing'
print("   ✅ Trialing subscription found")

# Past due within grace period
result = get_active_subscription(tenant_past_due, host_db, grace_period_days=3)
assert result is not None, "Should find past_due subscription within grace"
assert result.status == 'past_due'
print("   ✅ Past due subscription within grace period found")

# Canceled subscription
result = get_active_subscription(tenant_canceled, host_db)
assert result is None, "Should NOT find canceled subscription"
print("   ✅ Canceled subscription correctly not returned")

# No subscription
result = get_active_subscription(tenant_no_sub, host_db)
assert result is None, "Should NOT find subscription for tenant without one"
print("   ✅ No subscription correctly handled")

print("\n✅ All get_active_subscription tests PASSED!")

In [ ]:
# Test has_active_subscription
print("🧪 Testing has_active_subscription...")

assert has_active_subscription(tenant_active, host_db) == True
assert has_active_subscription(tenant_trialing, host_db) == True
assert has_active_subscription(tenant_past_due, host_db) == True
assert has_active_subscription(tenant_canceled, host_db) == False
assert has_active_subscription(tenant_no_sub, host_db) == False

print("   ✅ All has_active_subscription tests PASSED!")

In [ ]:
# Test require_active_subscription
print("🧪 Testing require_active_subscription...")

# Should return None for active subscription (allow access)
result = require_active_subscription(tenant_active, host_db)
assert result is None, "Should allow access for active subscription"
print("   ✅ Active subscription allows access")

# Should return 402 for no subscription
result = require_active_subscription(tenant_no_sub, host_db)
assert result is not None, "Should deny access for no subscription"
assert result.status_code == 402
print("   ✅ No subscription returns 402")

# Should redirect if redirect_url provided
result = require_active_subscription(tenant_canceled, host_db, redirect_url='/pricing')
assert result is not None
assert result.status_code == 303  # Redirect
print("   ✅ Canceled subscription redirects to pricing")

print("\n✅ All require_active_subscription tests PASSED!")

In [ ]:
# Test get_subscription_status
print("🧪 Testing get_subscription_status...")

# Active subscription status
status = get_subscription_status(tenant_active, host_db)
assert status['has_subscription'] == True
assert status['status'] == 'active'
assert status['plan_tier'] == 'monthly'
assert status['is_trial'] == False
print("   ✅ Active subscription status correct")

# Trialing subscription status
status = get_subscription_status(tenant_trialing, host_db)
assert status['has_subscription'] == True
assert status['status'] == 'trialing'
assert status['is_trial'] == True
assert status['trial_ends_at'] is not None
print("   ✅ Trialing subscription status correct")

# Past due status
status = get_subscription_status(tenant_past_due, host_db)
assert status['has_subscription'] == True
assert status['in_grace_period'] == True
print("   ✅ Past due subscription shows grace period")

# No subscription status
status = get_subscription_status(tenant_no_sub, host_db)
assert status['has_subscription'] == False
assert status['status'] == 'none'
print("   ✅ No subscription status correct")

# Canceled subscription status
status = get_subscription_status(tenant_canceled, host_db)
assert status['has_subscription'] == False
assert status['status'] == 'canceled'
assert status['cancel_at_period_end'] == True
print("   ✅ Canceled subscription status correct")

print("\n✅ All get_subscription_status tests PASSED!")

## Feature Gating Tests

In [ ]:
print("🧪 Testing Feature Gating...")

# Create yearly subscription for feature tests
tenant_yearly = 'tnt_yearly_test'
sub_yearly = Subscription(
    id=gen_id(),
    tenant_id=tenant_yearly,
    stripe_sub_id='sub_yearly_123',
    stripe_cust_id='cus_yearly',
    plan_tier='yearly',
    status='active',
    current_period_end=(datetime.utcnow() + timedelta(days=365)).isoformat(),
    payment_type='subscription',
    created_at=timestamp(),
)
host_db.subscriptions.insert(sub_yearly)
host_db.commit()

# Test tier hierarchy
assert TIER_HIERARCHY['free'] < TIER_HIERARCHY['monthly']
assert TIER_HIERARCHY['monthly'] < TIER_HIERARCHY['yearly']
assert TIER_HIERARCHY['yearly'] < TIER_HIERARCHY['enterprise']
print("   ✅ Tier hierarchy is correct")

# Yearly plan should have access to all features
assert check_feature_access(tenant_yearly, 'advanced_analytics', host_db=host_db) == True
assert check_feature_access(tenant_yearly, 'exports', host_db=host_db) == True
assert check_feature_access(tenant_yearly, 'api_access', host_db=host_db) == True
print("   ✅ Yearly plan has access to all features")

# Monthly plan should NOT have access to yearly features
assert check_feature_access(tenant_active, 'advanced_analytics', host_db=host_db) == False
assert check_feature_access(tenant_active, 'api_access', host_db=host_db) == True  # monthly feature
print("   ✅ Monthly plan feature access is correct")

# No subscription should only have free features
assert check_feature_access(tenant_no_sub, 'basic_features', host_db=host_db) == True
assert check_feature_access(tenant_no_sub, 'api_access', host_db=host_db) == False
print("   ✅ No subscription only has free features")

print("\n✅ All Feature Gating tests PASSED!")

In [ ]:
# Test require_feature_access
print("🧪 Testing require_feature_access...")

# Yearly plan accessing yearly feature - should allow
result = require_feature_access(tenant_yearly, 'advanced_analytics', host_db=host_db)
assert result is None, "Should allow yearly plan to access yearly feature"
print("   ✅ Yearly plan allowed for yearly feature")

# Monthly plan accessing yearly feature - should deny
result = require_feature_access(tenant_active, 'advanced_analytics', host_db=host_db)
assert result is not None
assert result.status_code == 403
print("   ✅ Monthly plan denied for yearly feature (403)")

# With redirect
result = require_feature_access(
    tenant_active, 'exports', 
    host_db=host_db, 
    redirect_url='/upgrade'
)
assert result is not None
assert result.status_code == 303
print("   ✅ Feature denied with redirect works")

print("\n✅ All require_feature_access tests PASSED!")

## Webhook Handler Tests

In [ ]:
print("🧪 Testing Webhook Handlers...")

# Reset service to get fresh instance
reset_stripe_service()

# Create service with test config
config = StripeConfig.from_env()
service = StripeService(config, host_db)

# Test checkout.session.completed for subscription
checkout_event = {
    'type': 'checkout.session.completed',
    'data': {
        'object': {
            'id': 'cs_webhook_test',
            'mode': 'subscription',
            'subscription': 'sub_webhook_test',
            'customer': 'cus_webhook_test',
            'metadata': {
                'tenant_id': 'tnt_webhook_test',
                'user_email': 'webhook@test.com',
                'plan_type': 'monthly',
            }
        }
    }
}

# Mock get_subscription to avoid real API call
with patch.object(service, 'get_subscription') as mock_get:
    mock_get.return_value = {
        'status': 'trialing',
        'current_period_end': int((datetime.utcnow() + timedelta(days=30)).timestamp()),
        'trial_end': int((datetime.utcnow() + timedelta(days=30)).timestamp()),
    }
    
    result = service.handle_event(checkout_event)
    assert result['status'] == 'success'
    print("   ✅ checkout.session.completed (subscription) handled")

# Verify subscription was created
sub = get_active_subscription('tnt_webhook_test', host_db)
assert sub is not None
assert sub.stripe_sub_id == 'sub_webhook_test'
assert sub.payment_type == 'subscription'
print("   ✅ Subscription record created in database")

In [ ]:
# Test checkout.session.completed for one-time payment
one_time_event = {
    'type': 'checkout.session.completed',
    'data': {
        'object': {
            'id': 'cs_onetime_test',
            'mode': 'payment',
            'customer': 'cus_onetime_test',
            'amount_total': 4999,
            'metadata': {
                'tenant_id': 'tnt_onetime_test',
                'user_email': 'onetime@test.com',
                'product_name': 'Premium Report',
                'amount_cents': '4999',
            }
        }
    }
}

result = service.handle_event(one_time_event)
assert result['status'] == 'success'
assert 'One-time' in result['message']
print("   ✅ checkout.session.completed (one-time) handled")

# Verify one-time payment was recorded
payments = host_db.subscriptions(
    where="tenant_id = 'tnt_onetime_test' AND payment_type = 'one_time'"
)
assert len(payments) == 1
assert payments[0].amount_cents == 4999
assert payments[0].product_name == 'Premium Report'
print("   ✅ One-time payment record created in database")

In [ ]:
# Test invoice.payment_succeeded
payment_success_event = {
    'type': 'invoice.payment_succeeded',
    'data': {
        'object': {
            'id': 'inv_success',
            'subscription': 'sub_webhook_test',
        }
    }
}

result = service.handle_event(payment_success_event)
assert result['status'] == 'success'
print("   ✅ invoice.payment_succeeded handled")

# Verify status updated to active
sub = service._find_subscription_by_stripe_id('sub_webhook_test')
assert sub.status == 'active'
print("   ✅ Subscription status updated to active")

In [ ]:
# Test invoice.payment_failed
payment_failed_event = {
    'type': 'invoice.payment_failed',
    'data': {
        'object': {
            'id': 'inv_failed',
            'subscription': 'sub_webhook_test',
        }
    }
}

result = service.handle_event(payment_failed_event)
assert result['status'] == 'success'
print("   ✅ invoice.payment_failed handled")

# Verify status updated to past_due
sub = service._find_subscription_by_stripe_id('sub_webhook_test')
assert sub.status == 'past_due'
print("   ✅ Subscription status updated to past_due")

In [ ]:
# Test customer.subscription.deleted
sub_deleted_event = {
    'type': 'customer.subscription.deleted',
    'data': {
        'object': {
            'id': 'sub_webhook_test',
        }
    }
}

result = service.handle_event(sub_deleted_event)
assert result['status'] == 'success'
print("   ✅ customer.subscription.deleted handled")

# Verify status updated to canceled
sub = service._find_subscription_by_stripe_id('sub_webhook_test')
assert sub.status == 'canceled'
assert sub.cancel_at_period_end == True
print("   ✅ Subscription status updated to canceled")

In [ ]:
# Test unhandled event type
unknown_event = {
    'type': 'unknown.event.type',
    'data': {'object': {}}
}

result = service.handle_event(unknown_event)
assert result['status'] == 'ignored'
print("   ✅ Unknown event type correctly ignored")

print("\n✅ All Webhook Handler tests PASSED!")

## Signature Verification Tests

In [ ]:
print("🧪 Testing Signature Verification...")

# In development mode, should bypass verification
dev_config = StripeConfig(
    secret_key='sk_test',
    webhook_secret=None,  # No webhook secret
    is_development=True,
)
dev_service = StripeService(dev_config, host_db)

payload = json.dumps({'type': 'test'}).encode('utf-8')
result = dev_service.verify_signature(payload, '')
assert result is not None
assert result['type'] == 'test'
print("   ✅ Development mode bypasses signature verification")

# Invalid JSON should fail
bad_payload = b'not valid json'
result = dev_service.verify_signature(bad_payload, '')
assert result is None
print("   ✅ Invalid JSON returns None")

print("\n✅ All Signature Verification tests PASSED!")

## Singleton Pattern Tests

In [ ]:
print("🧪 Testing Singleton Pattern...")

# Reset singleton
reset_stripe_service()

# First call creates instance
service1 = get_stripe_service()
assert service1 is not None

# Second call returns same instance
service2 = get_stripe_service()
assert service1 is service2
print("   ✅ Singleton pattern works correctly")

# Reset and create with custom config
reset_stripe_service()
custom_config = StripeConfig(
    secret_key='sk_custom',
    trial_days=14,
)
service3 = get_stripe_service(custom_config)
assert service3.config.trial_days == 14
print("   ✅ Custom config on first call works")

print("\n✅ All Singleton Pattern tests PASSED!")

In [ ]:
#| hide
# Cleanup
cleanup_test_db()
print("\n🎉 All utils_stripe tests completed successfully!")